# Hysteresis Loops Fitting Speed Measurements Benchmarking

In [ ]:
import sys
sys.path.append('../../')

In [ ]:
import numpy as np
import h5py
import time

import numpy as np
import h5py
import time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from src.m3_learning.optimizers.AdaHessian import AdaHessian
from src.m3_learning.util.preprocessing import global_scaler
from src.m3_learning.be.processing import loop_lsqf, loop_fitting_function_tf, loop_fitting_function_torch
from src.m3_learning.optimizers.TRPCGOptimizerv2 import TRPCGOptimizerv2

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import (Dense, Conv1D, Flatten, MaxPooling1D, Activation)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam

## Loads data

In [ ]:
# Sets path to file
path = r"./"

# Opens the data file2
h5_f = h5py.File(path + "data_file.h5", "r+")

# number of pixels in the image
num_pix = h5_f["Measurement_000"].attrs["num_pix"]

num_pix_1d = int(np.sqrt(num_pix))

# number of DC voltage steps
voltage_steps = h5_f["Measurement_000"].attrs["num_udvs_steps"]

proj_nd_shifted = loop_lsqf(h5_f)
proj_nd_shifted_transposed = np.transpose(proj_nd_shifted,(1,0,2,3))

# getting parameters for the hysteresis loops
params = np.array(h5_f['params_hysteresis'][:])
params_names = ['a_0', 'a_1', 'a_2', 'a_3', 'a_4', 'b_0', 'b_1', 'b_2', 'b_3']

# voltage vector
V = np.swapaxes(np.atleast_2d(h5_f['Measurement_000']['Channel_000']['UDVS'][::2][:, 1][24:120]), 0, 1).astype(np.float64)

# to set up a type of loop_fitting function to use. Possible options: ['9 parameters', '13 parameters']
func_type = '9 parameters'

# retrieve results
real_loops = np.array(h5_f['real_loops_hysteresis'][:])
unscaled_param_trust = np.array(h5_f['predictions_hysteresis_trustregcg'][:])
unscaled_param_adam = np.array(h5_f['predictions_hysteresis_adam'][:])

## Preprocess data

In [ ]:
real_loops_scaler = global_scaler()
real_scaled_loops = real_loops_scaler.fit_transform(real_loops).astype(np.float64)

real_parms_scaler = StandardScaler()
real_parms_scaled = real_parms_scaler.fit_transform(params)

# getting mean and std of parameters
params_mean = real_parms_scaler.mean_
params_std = np.sqrt(real_parms_scaler.var_)

data_mean = real_loops_scaler.mean.astype(np.float64)
data_std = real_loops_scaler.std.astype(np.float64)

## Benchmarking (Adam and AdaHessian)

In [ ]:
manual_seed = np.arange(1, 11, 1)
optimizers = [torch.optim.Adam, AdaHessian]
optimizers_name = ["ADAM", "ADAHESSIAN"]

In [ ]:
for k, optim in enumerate(optimizers):
  with open(f'{optimizers_name[k]}_Hysteresis_speed.txt', 'w') as file:
    for seed in manual_seed:
      file.write(f'MANUAL SEED: {seed}\n')
      
      for n in range(6, 11):
        torch.manual_seed(seed)
        np.random.seed(seed)
        torch.cuda.empty_cache() 
        
        class Pie_Model(nn.Module):
          def __init__(self):
              super().__init__()

              # Input block of 1d convolution
              self.hidden_x1 = nn.Sequential(
                  nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  
                  nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
              )

              # Flatten layer
              self.flatten_layer = nn.Flatten()
              
              # Final embedding block - Output 9 values - linear
              self.hidden_embedding = nn.Sequential(
                  nn.Linear(16, 9),
                  nn.LeakyReLU(),
              )

          def forward(self, x, n=-1):
            x = torch.unsqueeze(x, 1)
            x = self.hidden_x1(x)
            encoded = self.flatten_layer(x)
            embedding = self.hidden_embedding(encoded)
            unscaled_param = embedding*torch.tensor(params_std).cuda() \
                                    + torch.tensor(params_mean).cuda()
            scaled_loops = (loop_fitting_function_torch(func_type, V, unscaled_param) - torch.tensor(data_mean).cuda()) / torch.tensor(data_std).cuda()
            return scaled_loops

        
        model = Pie_Model().cuda().double()

        loss_func = torch.nn.MSELoss()
        batch_size = 2**n
        optimizer = AdaHessian(model.parameters(), lr=0.2) #0.1

        train_dataloader = DataLoader(real_scaled_loops, batch_size=batch_size)

        epochs = 1000000

        file.write(f'Traning with batch size = {batch_size}\n')
        start_train = time.time()
            
        for epoch in range(epochs):
          start_time = time.time()

          train_loss = 0.
          total_num = 0

          model.train()

          for train_batch in train_dataloader:
              
            pred = model(train_batch.double().cuda())

            optimizer.zero_grad()

            loss = loss_func(train_batch.double().cuda(), pred)
            loss.backward(create_graph=True)
            train_loss += loss.item() * pred.shape[0]
            total_num += pred.shape[0]

            optimizer.step()

          train_loss /= total_num
          torch.save(model, f'./Trained Models/Loops/model_{optimizers_name[k]}_bs_{batch_size}_seed_{seed}.pt')
          torch.save(model.state_dict(), f'./Trained Models/Loops/model_{optimizers_name[k]}_bs_{batch_size}_seed_{seed}.pth')

          if (time.time() - start_train) > 300:
            file.write('Training time: ' + str(time.time() - start_train) + ' seconds\n')
            file.write('Number of epochs: ' + str(epoch) + '\n')
            break
          
        class Pie_Model(nn.Module):
          def __init__(self):
              super().__init__()

              # Input block of 1d convolution
              self.hidden_x1 = nn.Sequential(
                  nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  
                  nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
                  nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                  nn.LeakyReLU(),
                  nn.MaxPool1d(kernel_size=2, padding=1),
              )

              # Flatten layer
              self.flatten_layer = nn.Flatten()
              
              # Final embedding block - Output 9 values - linear
              self.hidden_embedding = nn.Sequential(
                  nn.Linear(16, 9),
                  nn.LeakyReLU(),
              )

          def forward(self, x, n=-1):
            x = torch.unsqueeze(x, 1)
            x = self.hidden_x1(x)
            encoded = self.flatten_layer(x)
            embedding = self.hidden_embedding(encoded)
            unscaled_param = embedding*torch.tensor(params_std).cuda() \
                                    + torch.tensor(params_mean).cuda()
            return unscaled_param
        
        model_parameters = Pie_Model().cuda().double()
        model_parameters = torch.load(f'./Trained Models/Loops/model_{optimizers_name[k]}_bs_{batch_size}_seed_{seed}.pt')

        # prediction of parameters
        batch_size = 100000
        train_dataloader = DataLoader(real_scaled_loops, batch_size=batch_size)

        num_elements = len(train_dataloader.dataset)
        num_batches = len(train_dataloader)
        test_pred_params = torch.zeros_like(torch.tensor(params))

        start_time_inference = time.time()

        for i, train_batch in enumerate(train_dataloader):
          start = i*batch_size
          end = start + batch_size

          if i == num_batches - 1:
            end = num_elements

          pred_batch = model_parameters(train_batch.double().cuda())
          test_pred_params[start:end] = pred_batch.cpu().detach()

          del pred_batch
          del train_batch
          torch.cuda.empty_cache()
        
        file.write(f'{optimizers_name[k]} Inference time: ' + str((time.time() - start_time_inference) / real_scaled_loops.shape[0]) + ' seconds\n')

        scaled_loops_adahessian = (loop_fitting_function_torch(func_type, V, test_pred_params.cuda()) - torch.tensor(data_mean).cuda()) / torch.tensor(data_std).cuda()
        scaled_params_adahessian = (test_pred_params.cuda() - torch.tensor(params_mean).cuda()) / torch.tensor(params_std).cuda()

        mse_loops_adahessian = mean_squared_error(scaled_loops_adahessian.cpu().detach(), torch.tensor(real_scaled_loops))
        mse_params_adahessian = mean_squared_error(scaled_params_adahessian.cpu().detach(), torch.tensor(real_parms_scaled))

        file.write(f'MSE of hysteresis loops with {optimizers_name[k]}: ' + str(np.mean(mse_loops_adahessian)) + '\n')
        file.write('Params Predictions MSE (scaled): ' + str(np.mean(mse_params_adahessian)) + '\n')
        file.write('-----------------------------\n')
        file.write('\n')

## Benchmarking (Trust-Region CG)

In [ ]:
ACTIVATION = tf.nn.leaky_relu

def Conv1D_Block(X, time_step, kernel_size):
    x = Conv1D(time_step, kernel_size, padding='same')(X)
    x = Activation(ACTIVATION)(x)      
    return x

def Conv1D_Pie(kernel_size = 3, n_step = 96):
    X_input = Input(shape=(n_step, 1))
    x = X_input
    
    x = Conv1D_Block(x, 16, kernel_size)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D_Block(x, 32, kernel_size)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D_Block(x, 16, kernel_size)
    x = MaxPooling1D(2, padding='same')(x)
    
    time_step = 8 
    
    x = Conv1D_Block(x, time_step, kernel_size)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D_Block(x, time_step, kernel_size)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D_Block(x, time_step, kernel_size)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D_Block(x, time_step, kernel_size)
    x = MaxPooling1D(2, padding='same')(x)
    
    encoded = Flatten()(x)
    
    embedding = Dense(9, activation='linear')(encoded)
    embedding = tf.cast(embedding, dtype='float64')
    
    unscaled_param = tf.add(tf.multiply(embedding, tf.convert_to_tensor(params_std)),\
                            tf.convert_to_tensor(params_mean))
    scaled_loops = tf.divide(tf.subtract(loop_fitting_function_tf(func_type, V, unscaled_param), \
                            tf.convert_to_tensor(data_mean)), tf.convert_to_tensor(data_std))

    model = Model(X_input, scaled_loops, name = 'Convolutional_1D')
    
    return model

In [ ]:
with open('TRUST-REGION-CG_Hysteresis_speed.txt', 'w') as file:    
    for seed in manual_seed:
        file.write(f'MANUAL SEED: {seed}\n')
        tf.random.set_seed(seed)  
        
        for n in range(6, 11):
            batch_size = 2**n
            tf.keras.backend.set_floatx('float64')
            model = Conv1D_Pie()
            model.compile(optimizer=Adam(0.00001), loss='mse')

            FVAL = []
            earlyPredictor = tf.keras.Model(model.inputs,model.layers[23].output)
            embedding = real_parms_scaled
            unscaled_param = tf.add(tf.multiply(embedding, tf.convert_to_tensor(params_std)),\
                                tf.convert_to_tensor(params_mean))
            scaled_loops_ = tf.divide(tf.subtract(loop_fitting_function_tf(func_type, V, unscaled_param), \
                                tf.convert_to_tensor(data_mean)), tf.convert_to_tensor(data_std))
            
            file.write(f'Traning with batch size = {batch_size}\n')

            cgopttol = 1e-7
            c0tr = 0.2
            c1tr = 0.25
            c2tr = 0.75  # when to accept
            t1tr = 0.75
            t2tr = 2.0
            radius_max = 5.0  # max radius
            radius_initial = 1.0
            radius = radius_initial

            optimizer = TRPCGOptimizerv2(model,radius_initial,0)

            allsamples=[i for i in range(num_pix)]

            st = time.time()
            for epoch in range(1000):

                np.random.shuffle(allsamples)

                BS = batch_size
                for it in range(num_pix//BS):

                    x = real_scaled_loops[allsamples[it*BS:(it+1)*BS]]
                    y = real_scaled_loops[allsamples[it*BS:(it+1)*BS]]


                    loss, d, rho, update, cg_iter, cg_term, loss_grad, norm_d, numerator, denominator, rad = \
                    optimizer.step(x,y)
                    
                    parm_pred = earlyPredictor.predict(real_scaled_loops)
                    embedding = parm_pred
                    unscaled_param = tf.add(tf.multiply(embedding, tf.convert_to_tensor(params_std)),\
                                        tf.convert_to_tensor(params_mean))
                    scaled_loops_DNN = tf.divide(tf.subtract(loop_fitting_function_tf(func_type, V, unscaled_param), \
                                        tf.convert_to_tensor(data_mean)), tf.convert_to_tensor(data_std))

                    
                    err = tf.reduce_mean(tf.abs(scaled_loops_DNN - real_scaled_loops)).numpy()
                    
                    FVAL.append([loss.numpy(),err])
                    if optimizer.radius < 1e-15:
                        break

                if(time.time() - st > 300):
                    file.write('Training time: ' + str(time.time() - st) + ' seconds\n')
                    file.write('Number of epochs: ' + str(epoch) + '\n')
                    break

            model.save(f'./Trained Models/Loops/model_bs{batch_size}')
            unscaled_param_trust = tf.identity(unscaled_param)
            scaled_params_trust = tf.identity(embedding)

            start_time_inference = time.time()
            earlyPredictor.predict(real_scaled_loops)
            file.write('Trust Region Inference time: ' + str((time.time() - start_time_inference) / real_scaled_loops.shape[0]) + ' seconds\n')

            scaled_loops_DNN_trust = scaled_loops_DNN
            real_scaled_loops_trust = real_scaled_loops
            scaled_loops_trust = scaled_loops_
            
            start_time_inference = time.time()
            earlyPredictor.predict(real_scaled_loops)
        
            errors = tf.reduce_mean(tf.abs(scaled_loops_DNN_trust - real_scaled_loops) + tf.abs(scaled_loops_DNN - real_scaled_loops), 1)

            mse = tf.keras.losses.MeanSquaredError()
            trust_region_error = mse(scaled_loops_DNN_trust, real_scaled_loops).numpy()

            mae = tf.keras.losses.MeanAbsoluteError()
            trust_region_mae = mae(scaled_loops_DNN_trust, real_scaled_loops).numpy()

            errors = np.asarray(errors)
            trust_region_error = np.asarray(trust_region_error)

            unscaled_loops_lsqf = loop_fitting_function_tf(func_type, V, params)
            scaled_loops_lsqf = tf.divide(tf.subtract(loop_fitting_function_tf(func_type, V, params), \
                                        tf.convert_to_tensor(data_mean)), tf.convert_to_tensor(data_std))
            
            mse_loops_trust = np.mean(np.square((scaled_loops_DNN_trust - real_scaled_loops)), 1)
            mse_loops_lsqf = np.mean(np.square((scaled_loops_lsqf - real_scaled_loops)), 1)
            highest_loops_trust = (-mse_loops_trust).argsort()[:]
            highest_loops_lsqf = (-mse_loops_lsqf).argsort()[:]
            file.write('MSE of hysteresis loops with Trust Region CG: ' + str(np.mean(mse_loops_trust)) + '\n')
            file.write('MSE of hysteresis loops with LSQF: ' + str(np.mean(mse_loops_lsqf)) + '\n')
            file.write('Params Predictions MSE (scaled): ' + str(mse(scaled_params_trust, real_parms_scaled)) + '\n')
            file.write('Params Predictions MSE (unscaled): ' + str(mse(unscaled_param_trust, params)) + '\n')
            file.write('-----------------------------\n')
            file.write('\n\n')